EL PLANTEAMIENTO CON RESPECTO A LA PRIMERA VERSIÓN ES DIFERENTE:
A LA VISTA DE LOS ERRORES QUE SE PRODUCEN CON LA DESCARGA DEL TOTAL RETURN POR PARTES PRINCIPALMENTE DE LOS ÍNDICES DE RENTA FIJA, LO QUE SE HACE EN ESTE ALGORITMO ES DESCARGAR TODA LA SERIE TEMPORAL DE PRECIOS, ALINEARLAS TEMPORALMENTE Y POSTERIORMENTE CALCULAR LAS CARTERAS Y EL PERFORMANCE.
DE HECHO, ESTA VERSIÓN SE HA PLANTEADO DEBIDO A QUE ALGUNOS ÍNDICES NO ESTÁN REPRESENTADOS EN PORT <GO> Y LOS RESULTADOS QUE SE OBTIENEN NO SON REALES.

In [1]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
import bqviz as bqv
from scipy.optimize import minimize
from dateutil.relativedelta import relativedelta
import bqport
from bqport import PositionType
from sklearn import linear_model

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

equity_idx = ['ACWI US Equity', 'EFA US Equity', 'XMEU GY Equity', 'VDEV LN Equity', 'IWVL LN Equity','IEMG US Equity','XRSU LN Equity']
fi_idx = ['FLOA LN Equity', 'IGIL LN Equity', 'CBU7 LN Equity']

etf_list = equity_idx + fi_idx

In [2]:
# INPUTS ###############

index_to_replicate = 'HFRIFOF Index'

start_date = datetime.datetime(2006,1,1)
end_date = datetime.datetime(2019,1,1)
curncy = 'USD'
rotation = 1 #months

In [3]:
#The fund inception of some ETFs it's too close to crrent days, so to make the analysis we will asume that these ETFs replicate exactly the underlying indexes.
#Due to this hypothesis, the time series are going to be analyzed with the underlying indexes.

etf_under_idx = d.fund_benchmark()

request = bql.Request(u.list(etf_list), {'UND_IDX': etf_under_idx})
res = bq.execute(request)
etf_under_idx_list = [undr +' Index' for undr in list(res[0].df()['UND_IDX'])]

#For the time period specified, the underlying indexes of VDEV LN Equity, ACIM US Equity and XRSU LN Equity there aren't "longer enough (they start later)
#We asume that the Tracking Error between these indexes and the original is quite low

for i in range(len(etf_under_idx_list)):
    if etf_under_idx_list[i] == 'AWNT04U Index':
        etf_under_idx_list[i] = 'FTAW01 Index'
    elif etf_under_idx_list[i] == 'M1WDIM Index':
        etf_under_idx_list[i] = 'MXWDIM Index'
    elif etf_under_idx_list[i] == 'RU20N30U Index':
        etf_under_idx_list[i] = 'RTY Index'

In [4]:
#Optimization functions
def f_obj(x):
    return sum((np.matmul(A, x) - y)**2)

def constraint_total_weight_ub(x):  #The sum of all weights must be 1
    return 1 - np.sum(x)

def compute_port_vs_bench_difference (df_port, df_group_price_series, df_align_price_series, df_index_price_series):
    
    #Securities don't follow the same order in different time periods
    df_port = df_port.sort_values(by=['date', 'security'])

    #Add to the portfolio df an auxiliar date 
    year_list = pd.DatetimeIndex(df_port['date']).year.tolist()
    month_list = pd.DatetimeIndex(df_port['date']).month.tolist()
    date_list = []
    for j in range(len(year_list)):
        date_list += [str(year_list[j]) + '-' + str(month_list[j])]
    df_port['aux_date'] = date_list  

    #Get the price and consecuently the number of shares for each moment of the rebalancing date
    list_reb_dates = df_port['aux_date'].unique().tolist()

    price_0_list = []
    for i_date in list_reb_dates:
        aux_port_df = df_port.loc[df_port['aux_date'].isin([i_date])]
        price_0_list += df_group_price_series.loc[df_group_price_series['aux_date'].isin([i_date])][aux_port_df['security'].tolist()].values[0].tolist()
    df_port['Price'] = price_0_list
    df_port['N_Assets_type'] = df_port['weight']/df_port['Price']

    #For the first date the calculated number of shares it's OK, but for the rest of the dates it must be take into account the value of the portfolio in that moment
    aux_df_real_n_assets = df_port.loc[df_port['aux_date'].isin([list_reb_dates[0]])][['aux_date', 'N_Assets_type']]

    for i_prev_date,i_date in zip(list_reb_dates[:-1], list_reb_dates[1:]):
        n_assets = np.array(aux_df_real_n_assets.loc[aux_df_real_n_assets['aux_date'].isin([i_prev_date])]['N_Assets_type'].tolist())
        cur_price = np.array(df_port.loc[df_port['aux_date'].isin([i_date])]['Price'].tolist())
        val = sum(n_assets*cur_price)
        cur_n_assets = val*np.array(df_port.loc[df_port['aux_date'].isin([i_date])]['weight'].tolist())/cur_price

        aux_df_real_n_assets = aux_df_real_n_assets.append(pd.DataFrame(data={'aux_date': [i_date]*len(cur_n_assets), 'N_Assets_type': cur_n_assets}), ignore_index = True)

    df_port['Real_N_assets'] = aux_df_real_n_assets['N_Assets_type'].tolist()

    #Put the portfolio df with the number of shares in each period of time in the same format as the price series df
    port_df_pivot = df_port.pivot(index = 'aux_date', columns = 'security', values = 'Real_N_assets').reset_index()

    n_assets_type_table = pd.merge(port_df_pivot, df_align_price_series['aux_date'].to_frame(), how='left', on=['aux_date', 'aux_date']).fillna(method='ffill')

    #Compute the portfolio value
    port_val = df_align_price_series.copy()
    port_val = port_val.loc[port_val['aux_date'].isin(n_assets_type_table['aux_date'].unique())]
    for asset in etf_under_idx_list:
        port_val[asset] = np.array(port_val[asset].tolist()) * np.array(n_assets_type_table[asset].tolist())

    port_val['Total_Abs'] = port_val[etf_under_idx_list].sum(axis=1)

    #Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
    def_series = df_index_price_series.copy()
    def_series = def_series.loc[def_series['aux_date'].isin(n_assets_type_table['aux_date'].unique())]
    aligned_port_vals = []
    for i_date in def_series['aux_date'].tolist():
        aligned_port_vals += [port_val.loc[port_val['aux_date'].isin([i_date])]['Total_Abs'].values[-1]]

    #Calculate the cumulated total return of both value series
    def_series['Port'] = aligned_port_vals
    def_series = def_series.dropna(subset = ['Port'])
    def_series[['Port', index_to_replicate]] = (def_series[['Port', index_to_replicate]].pct_change().fillna(0) + 1).cumprod() - 1

    #Sort Columns
    def_series = def_series[['DATE', 'Port', index_to_replicate, 'aux_date']]

    diff = def_series.iloc[len(def_series)-1]['Port'] - def_series.iloc[len(def_series)-1][index_to_replicate]

    return diff

In [5]:
#Price Aligneation ####################

#Get the whole price series por all the indexes of the ETFs
download_start_date = start_date - relativedelta(months=12)
download_end_date = end_date + relativedelta(months=rotation)

price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(u.list(etf_under_idx_list), {'Price': price})
res = bq.execute(request)
price_series = res[0].df().reset_index()

#Pivot the price series to get the df in a way that the rows are dates and columns indexes prices
align_price_series = price_series.pivot(index='DATE', columns='ID', values='Price').reset_index().dropna().reset_index().drop(['index'], axis=1)

#Add to the price series df an auxiliar date 
year_list = pd.DatetimeIndex(align_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(align_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
align_price_series['aux_date'] = date_list  

#Get the price series for HFRIFOF Index (the returned data is in a monthly basis)
price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(index_to_replicate, {'Price': price})
res = bq.execute(request)
index_price_series = res[0].df().reset_index()

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
index_price_series = index_price_series.drop(['ID', 'CURRENCY'], axis=1)
index_price_series = index_price_series.rename(index=str, columns={'Price': index_to_replicate})
year_list = pd.DatetimeIndex(index_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(index_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
index_price_series['aux_date'] = date_list

grouped_price_series = pd.DataFrame(data = {})

list_aux_dates = align_price_series['aux_date'].unique().tolist()
for i_date in list_aux_dates:
    aux_mat = align_price_series.loc[align_price_series['aux_date'].isin([i_date]), align_price_series.columns != 'DATE']
    i_idx = aux_mat.index.tolist()
    aux_mat = aux_mat.loc[i_idx[-1]].to_frame().T
    
    if i_date == list_aux_dates[0]:
        grouped_price_series = aux_mat
        
    else:
        grouped_price_series = grouped_price_series.append(aux_mat, ignore_index=True)

grouped_price_series = pd.merge(index_price_series, grouped_price_series, how = 'left', on=['aux_date', 'aux_date'])

index_universe = [index_to_replicate]+etf_under_idx_list

In [6]:
#Backtesting ############################

aux_date = start_date
while end_date >= aux_date:
    print(aux_date)
    
    aux_date_st = aux_date - relativedelta(months=12)

    #Compute the spread between the portfolio and the benchmark. This spread (the cumulative total return to the current point) is going to be applied in the in-sample cumulative total return
    #of the benchmark to shift the base. This is a way to correct the out-of-sample tracking error
    try:
        spread_port_bk = compute_port_vs_bench_difference(port_df, grouped_price_series, align_price_series, index_price_series)
    
    except:
        spread_port_bk = 0
    
    print('spread: ', spread_port_bk)
    
    test_data = grouped_price_series.loc[(grouped_price_series['DATE'] >= aux_date_st) & (grouped_price_series['DATE'] <= aux_date)]
    test_data = test_data.loc[:,((grouped_price_series.columns != 'aux_date') & (grouped_price_series.columns != 'DATE')).tolist()]

    #Calculate the return
    test_data = test_data.pct_change().fillna(0)    
    
#     #Calculate the cumulative total return
#     test_data = (test_data.pct_change().fillna(0) + 1).cumprod() - 1    
    
    #Optimzation of series without UCITS constraints to see which is the optimal allocation

    #Apply RANSAC to get rid off the outliers fron the sample
    A = []
    for i in range(len(test_data)):
        A += [test_data[etf_under_idx_list].iloc[i].values.tolist()]
    A = np.array(A)    

    y = test_data[index_to_replicate].tolist()
    y = np.array(y) - spread_port_bk
    
    ransac = linear_model.RANSACRegressor(max_trials=1000)
    ransac.fit(A, y)
    
    test_data = test_data.loc[ransac.inlier_mask_]
    
    #Apply to the filtered sample with RANSAC the optimization
    A = []
    for i in range(len(test_data)):
        A += [test_data[etf_under_idx_list].iloc[i].values.tolist()]
    A = np.array(A)    

    y = test_data[index_to_replicate].tolist()
    y = np.array(y) - spread_port_bk

    guess=np.zeros(len(etf_under_idx_list))

    bnds = [(0,1) for i in range(len(etf_under_idx_list))]
    results = minimize(fun = f_obj, x0 = guess, method='SLSQP', bounds = bnds, options={'disp': True}, constraints= [{'type':'eq','fun' : constraint_total_weight_ub}])  
    
    if aux_date == start_date:
        port_df = pd.DataFrame({'date': aux_date,'security': etf_under_idx_list, 'weight': results.x})
    else :
        aux_port_df = pd.DataFrame({'date': aux_date,'security': etf_under_idx_list, 'weight': results.x})
        port_df = pd.concat([port_df, aux_port_df])

    aux_date = aux_date + relativedelta(months=rotation)
    
port_df

2006-01-01 00:00:00
spread:  0
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.00022516487591
            Iterations: 8
            Function evaluations: 96
            Gradient evaluations: 8
2006-02-01 00:00:00
spread:  0.0
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000225337594448
            Iterations: 5
            Function evaluations: 60
            Gradient evaluations: 5
2006-03-01 00:00:00
spread:  -0.00547741468859
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000406056168753
            Iterations: 5
            Function evaluations: 60
            Gradient evaluations: 5
2006-04-01 00:00:00
spread:  -0.0107189613641
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000772161385461
            Iterations: 9
            Function evaluations: 108
            Gradient evaluations: 9
2006-05-01 

date        security        weight
0  2006-01-01  NDUEACWF Index  8.803162e-02
1  2006-01-01  NDDUEAFE Index  8.655077e-02
2  2006-01-01   NDDUE15 Index  8.236160e-02
3  2006-01-01    FTAW01 Index  8.852414e-02
4  2006-01-01    M1WOEV Index  6.812773e-02
5  2006-01-01  MIMUEMRN Index  3.930233e-19
6  2006-01-01       RTY Index  7.095230e-02
7  2006-01-01  BFU5TRUU Index  1.730684e-01
8  2006-01-01    BCIW1A Index  1.698976e-01
9  2006-01-01  IDCOT3TR Index  1.724859e-01
0  2006-02-01  NDUEACWF Index  9.531473e-02
1  2006-02-01  NDDUEAFE Index  8.836059e-02
2  2006-02-01   NDDUE15 Index  8.892079e-02
3  2006-02-01    FTAW01 Index  9.508972e-02
4  2006-02-01    M1WOEV Index  7.872274e-02
5  2006-02-01  MIMUEMRN Index  1.387779e-17
6  2006-02-01       RTY Index  5.114512e-02
7  2006-02-01  BFU5TRUU Index  1.677043e-01
8  2006-02-01    BCIW1A Index  1.634285e-01
9  2006-02-01  IDCOT3TR Index  1.713135e-01
0  2006-03-01  NDUEACWF Index  9.455044e-02
1  2006-03-01  NDDUEAFE Index  9.659759e-02
2  2006-03-01   NDDUE15 Index  9.344360e-02
3  2006-03-01    FTAW01 Index  9.447564e-02
4  2006-03-01    M1WOEV Index  8.680598e-02
5  2006-03-01  MIMUEMRN Index  3.895048e-02
6  2006-03-01       RTY Index  5.180811e-02
7  2006-03-01  BFU5TRUU Index  1.483379e-01
8  2006-03-01    BCIW1A Index  1.441540e-01
9  2006-03-01  IDCOT3TR Index  1.508763e-01
..        ...             ...           ...
0  2018-11-01  NDUEACWF Index  0.000000e+00
1  2018-11-01  NDDUEAFE Index  1.860038e-17
2  2018-11-01   NDDUE15 Index  0.000000e+00
3  2018-11-01    FTAW01 Index  1.409586e-16
4  2018-11-01    M1WOEV Index  0.000000e+00
5  2018-11-01  MIMUEMRN Index  0.000000e+00
6  2018-11-01       RTY Index  2.215274e-01
7  2018-11-01  BFU5TRUU Index  7.784726e-01
8  2018-11-01    BCIW1A Index  0.000000e+00
9  2018-11-01  IDCOT3TR Index  6.065319e-16
0  2018-12-01  NDUEACWF Index  7.394769e-02
1  2018-12-01  NDDUEAFE Index  0.000000e+00
2  2018-12-01   NDDUE15 Index  0.000000e+00
3  2018-12-01    FTAW01 Index  0.000000e+00
4  2018-12-01    M1WOEV Index  0.000000e+00
5  2018-12-01  MIMUEMRN Index  3.165870e-17
6  2018-12-01       RTY Index  1.488699e-01
7  2018-12-01  BFU5TRUU Index  7.771824e-01
8  2018-12-01    BCIW1A Index  0.000000e+00
9  2018-12-01  IDCOT3TR Index  0.000000e+00
0  2019-01-01  NDUEACWF Index  0.000000e+00
1  2019-01-01  NDDUEAFE Index  1.264274e-17
2  2019-01-01   NDDUE15 Index  0.000000e+00
3  2019-01-01    FTAW01 Index  0.000000e+00
4  2019-01-01    M1WOEV Index  0.000000e+00
5  2019-01-01  MIMUEMRN Index  2.486580e-17
6  2019-01-01       RTY Index  3.831350e-17
7  2019-01-01  BFU5TRUU Index  4.500010e-01
8  2019-01-01    BCIW1A Index  0.000000e+00
9  2019-01-01  IDCOT3TR Index  5.499990e-01

[1570 rows x 3 columns]

In [7]:
#Securities don't follow the same order in different time periods
port_df = port_df.sort_values(by=['date', 'security'])

#Add to the portfolio df an auxiliar date 
year_list = pd.DatetimeIndex(port_df['date']).year.tolist()
month_list = pd.DatetimeIndex(port_df['date']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
port_df['aux_date'] = date_list  

#Get the price and consecuently the number of shares for each moment of the rebalancing date
list_reb_dates = port_df['aux_date'].unique().tolist()

price_0_list = []
for i_date in list_reb_dates:
    aux_port_df = port_df.loc[port_df['aux_date'].isin([i_date])]
    price_0_list += grouped_price_series.loc[grouped_price_series['aux_date'].isin([i_date])][aux_port_df['security'].tolist()].values[0].tolist()
port_df['Price'] = price_0_list
port_df['N_Assets_type'] = port_df['weight']/port_df['Price']

#For the first date the calculated number of shares it's OK, but for the rest of the dates it must be take into account the value of the portfolio in that moment
aux_df_real_n_assets = port_df.loc[port_df['aux_date'].isin([list_reb_dates[0]])][['aux_date', 'N_Assets_type']]
    
for i_prev_date,i_date in zip(list_reb_dates[:-1], list_reb_dates[1:]):
    n_assets = np.array(aux_df_real_n_assets.loc[aux_df_real_n_assets['aux_date'].isin([i_prev_date])]['N_Assets_type'].tolist())
    cur_price = np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['Price'].tolist())
    val = sum(n_assets*cur_price)
    cur_n_assets = val*np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['weight'].tolist())/cur_price
    
    aux_df_real_n_assets = aux_df_real_n_assets.append(pd.DataFrame(data={'aux_date': [i_date]*len(cur_n_assets), 'N_Assets_type': cur_n_assets}), ignore_index = True)

port_df['Real_N_assets'] = aux_df_real_n_assets['N_Assets_type'].tolist()

#Put the portfolio df with the number of shares in each period of time in the same format as the price series df
port_df_pivot = port_df.pivot(index = 'aux_date', columns = 'security', values = 'Real_N_assets').reset_index()
n_assets_type_table = pd.merge(align_price_series['aux_date'].to_frame(), port_df_pivot, how='left', on=['aux_date', 'aux_date']).fillna(method='ffill')

#Compute the portfolio value
port_val = align_price_series.copy()
for asset in etf_under_idx_list:
    port_val[asset] = port_val[asset] * n_assets_type_table[asset]
    
port_val['Total_Abs'] = port_val[etf_under_idx_list].sum(axis=1)

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
def_series = index_price_series.copy()

aligned_port_vals = []
for i_date in def_series['aux_date'].tolist():
    aligned_port_vals += [port_val.loc[port_val['aux_date'].isin([i_date])]['Total_Abs'].values[-1]]

#Calculate the cumulated total return of both value series
def_series['Port'] = aligned_port_vals
def_series = def_series.dropna(subset = ['Port'])
def_series[['Port', index_to_replicate]] = (def_series[['Port', index_to_replicate]].pct_change().fillna(0) + 1).cumprod() - 1

#Sort Columns
def_series = def_series[['DATE', 'Port', index_to_replicate, 'aux_date']]

#Plot the result
plot_def_series = def_series.drop(['aux_date'], axis=1).set_index('DATE')
result = bqv.ComparisonPlot(plot_def_series)

# Display the plot
result.show()

In [8]:
# pd.pivot_table(port_df, values='weight', index=['date'],columns=['security'], aggfunc=np.sum).reset_index().to_excel('pesos_cartera_replica.xlsx')